# Коды обнаружения и исправления ошибок

В основе многих компьютерных технологий лежат не вполне устойчивые среды переноса и хранения информации, которые так и норовят поменять бит-другой во вверенных им данных. К таким ненадёжным средам относятся сети Wi-Fi и GSM, носители Flash, DVD, SSD и даже оперативная память. Почему же помехи не портят всем жизнь? Потому что данные обычно кодируются так, чтобы обнаруживать ошибки, а иногда ещё и исправлять их. В этой задаче вы познакомитесь с некоторыми алгебраическими принципами, лежащими в основе методов обнаружения и коррекции ошибок.

![](https://imgur.com/7g0fLMM.jpg)

Заранее обращаем ваше внимание на рисунок, изображающий процесс получения в $\mathbb{Z}_2[x]$ остатка от деления многочлена $x^{13}+ x^{11} + x^{10} + x^8 + x^6 + x^4 + x^2 + x + 1$ на $x^5 + x^2 + 1$ — остаток равен $x^4 + x^3 + x$. Каждый многочлен изображён вектором своих коэффициентов. Это представление может помочь вам в решении первых трёх подзадач.

*Технические замечания.* В этой задаче вам может понадобиться проводить вычисления в полях $\mathbb{Z}_p$, где $p$ — простое число. Вычисление обратного элемента для ненулевого $a$ из $\mathbb{Z}_p$ (т.е. такого $b \in \mathbb{Z}_p$, что $a\cdot b \equiv 1 \pmod{p}$) выполняется в Python 3.8 следующим образом: `b = pow(a, -1, p)`; если ваша версия Python более старая, можете воспользоваться подготовленной нами функцией `modinv`: https://pastebin.com/SmmQiLSx. Чтобы избежать неочевидных ошибок переполнения при выполнении вычислений в $\mathbb{Z}_p$, рекомендуем брать остаток от деления после *каждой* операции; например, выражение $\frac{4 + x^3}{a - b + 1}$ в $\mathbb{Z}_7$ вычисляется как `((4 + ((x*x)%7)*x%7) % 7 * pow(((a - b) % 7 + 1) % 7, -1, 7)) % 7`. В частности, для вычисления $x^n$ в $\mathbb{Z}_p$ не используйте код `x**n % p`, который приведёт к ошибкам переполнения при большом $n$.

**1. Коды обнаружения наличия ошибок.**
Предположим, что вам нужно переслать по каналу с помехами последовательность битов $b_1, b_2, \ldots, b_n$ ($b_i \in \{0,1\}$), по возможности обнаружив наличие искажений, если таковые будут. Самая распространённая схема состоит в вычислении по $b_1, b_2, \ldots, b_n$ 32-битового кода $c_1, c_2, \ldots, c_{32}$ ($c_i \in \{0,1\}$) и дописывании его в конец; затем сторона, принимающая биты $b_1, b_2, \ldots, b_n, c_1, c_2, \ldots, c_{32}$, тем же способом вычисляет по $b_1, b_2, \ldots, b_n$ код $c'_1, c'_2, \ldots, c'_{32}$ и сравнивает его с полученным кодом $c_1, c_2, \ldots, c_{32}$. Если коды не совпали, значит где-то вкралась ошибка, и тогда надо либо запросить данные снова, либо сообщить об ошибке.

Зафиксируем многочлен $g(x)$ степени $32$ в кольце $\mathbb{Z}_2[x]$. Самый популярный код проверки наличия ошибок вычисляет код $c_1, c_2, \ldots, c_{32}$ так, чтобы многочлен $c_1 x^{31} + c_2 x^{30} + \cdots + c_{32} = \sum_{i=1}^{32} c_i x^{32-i}$ являлся остатком от деления многочлена $b_1 x^{n+31} + b_2 x^{n+30} + \cdots + b_n x^{32} = x^{32} \sum_{i=1}^n b_i x^{n-i}$ на $g(x)$ в $\mathbb{Z}_2[x]$. Качество такого кода зависит от выбора $g(x)$. В этой подзадаче $g(x)$ дан вам массивом `g` длины 33 с элементами $g_0, g_1, \ldots, g_{32}$, так что $g(x) = \sum_{i=0}^{32} g_i x^{32-i}$ (обратите внимание, что если $g_0 = 1$, то $\deg g = 32$), и вам нужно дописать функцию, возвращающую код $c_1, c_2, \ldots, c_{32}$.  Заметим, что для этого и последующих заданий выбор числа 32 не принципиален и продиктован лишь тем, что по разным причинам в реальности это самый распространённый вариант; также иногда применяются коды размера 4, 5, 8, 16, 21, 24, 64 битов и другие.

In [ ]:
def integrity_code(data):
    '''32-битовый код проверки наличия ошибок для списка битов data'''
    g = [1, 0,0,0,0, 0,1,0,0, 1,1,0,0, 0,0,0,1, 0,0,0,1, 1,1,0,1, 1,0,1,1, 0,1,1,1]
    code = data + [0]*32
    for i in range(len(data)):
        if code[0] == 1:
            for j in range(33):
                code[j] = (code[j] + g[j]) % 2
        code = code[1:]
    return code

**2. Кучные изменения.**
Ясно, что как бы тщательно ни выбирался многочлен $g(x)$, в некоторых случаях ошибки заметить не удастся. Поэтому $g(x)$ подбирают так, чтобы хотя бы отлавливать типичные для данного приложения ошибки. 

Пусть $c_1,\ldots,c_{32}$ — это код, полученный по битам $b_1, \ldots, b_n$ с помощью некоторого фиксированного $g(x)$, как в предыдущем подзадании. Обозначим через $\tilde{b}_1, \ldots, \tilde{b}_n, \tilde{c}_1, \ldots, \tilde{c}_{32}$ последовательность, полученную из последовательности $b_1, \ldots, b_n, c_1, \ldots, c_{32}$ изменением битов в позициях $i_1, \ldots, i_m$, таких что $1 \le i_1 < \cdots < i_m \le n + 32$. Будем называть эти изменения *кучными*, если $i_m - i_1 < 32$, т.е. все изменения происходили на отрезке длины 32. Будем говорить, что изменения *обнаруживаются*, если код $c'_1, \ldots, c'_{32}$, вычисленный по $\tilde{b}_1, \ldots, \tilde{b}_n$ с помощью того же $g(x)$, не совпадает с $\tilde{c}_1, \ldots, \tilde{c}_{32}$. Для простоты ограничимся случаем, когда изменениям при передаче по каналу связи подвергается только часть $b_1, \ldots, b_n$, т.е. $i_m \le n$.<!-- (хотя всё нижеследующее будет верно, даже если не накладывать это ограничение). Обратите внимание, что изменениям может подвергаться не только часть $b_1, \ldots, b_n$, но и $c_1, \ldots, c_{32}$.--> 

Допишите функцию, которая на вход принимает многочлен $g(x)$ из $\mathbb{Z}_2[x]$ степени не больше 32, закодированный последовательностью нулей и единиц $g_0, g_1, \ldots, g_{32}$, так что $g(x) = \sum_{i=0}^{32} g_i x^{32-i}$ (здесь $g_0, g_1,\ldots$ могут быть нулями, так что степень $g$ может быть меньше $32$), и возвращает `True` тогда и только тогда, когда в любой последовательности $b_1, \ldots, b_n, c_1, \ldots, c_{32}$, в которой код $c_1, \ldots, c_{32}$ вычислен по $b_1, \ldots, b_n$ с помощью $g(x)$, любые кучные изменения, затрагивающие только часть $b_1,\ldots,b_n$, обнаруживаются. (Необязательное задание: докажите, что ограничение изменений только на часть $b_1,\ldots,b_n$ не существенно и те же условия гарантируют обнаружение вообще любых кучных изменений.)

*Подсказка.* Покажите, что в $\mathbb{Z}_2[x]$ остаток от деления суммы многочленов $f(x)$ и $h(x)$ на $g(x)$ равен сумме остатков от деления $f(x)$ на $g(x)$ и $h(x)$ на $g(x)$. Заметьте, что $x^{32} \sum_{i=1}^n \tilde{b}_i x^{n-i} = x^{32} \sum_{i=1}^n b_i x^{n-i} + x^{32} \sum_{j=1}^m x^{n-i_j}$.  Поэтому изменения в позициях $i_1, \ldots, i_m$ обнаруживаются тогда и только тогда, когда остаток от деления многочлена $x^{32} \sum_{j=1}^m x^{n-i_j}$ на $g(x)$ не равен нулю.

In [31]:
def g_detects_32bursts(g):
    return g[32] == 1 and g[0] == 1

**3. Подделка кода.** (*эта задача сложная, рекомендуем сначала решить все другие задачи*)
Код проверки наличия ошибок защищает от случайных помех и повреждений среды, а не от злонамеренной подделки информации. Чтобы убедиться в этом, допишите функцию, которая получает никак не связанные друг с другом списки битов `data` (длины не меньше 32) и `code` (длины ровно 32) и изменяет первые 32 бита `data[0]`,...,`data[31]` так, чтобы `code` был равен `integrity_code(data)`; значение `g` вам дано и оно такое же, как в функции `integrity_code` из подзадачи 1. Переборные решения не принимаются — ваш код должен менять биты «по-умному» и отрабатывать за секунду.

*Подсказка.* Пусть $b_1, \ldots, b_n$ — это последовательность `data[0]`, ..., `data[len(data)-1]`, где $n =$ `len(data)`. Рассмотрим многочлен $b(x) = x^{32}\sum_{i=1}^n b_i x^{n-i}$ из $\mathbb{Z}_2[x]$. Измененние бита $b_i$ равносильно прибавлению к $b(x)$ монома $x^{32+n-i}$ и поэтому изменение `data[0]`,...,`data[31]` равносильно прибавлению к $b(x)$ комбинации мономов $x^{32+n-1}, x^{32+n-2}, \ldots, x^{n}$. Обратите внимание, что операция взятия остатка от деления (т.е. $b(x) \mapsto b(x) \bmod g(x)$ при фиксированном $g(x)$) является линейным отображением из $\mathbb{Z}_2[x]$ в подпространство многочленов над $\mathbb{Z}_2$ со степенью меньше 32. Воспользуйтесь этими рассуждениями и вашим решением предыдущей подзадачи, чтобы получить алгоритм.

In [32]:
def falsify_integrity_code(data, code):
    '''Подделать первые 32 бита data, чтобы код стал равен code'''
    g = [1, 0,0,0,0, 0,1,0,0, 1,1,0,0, 0,0,0,1, 0,0,0,1, 1,1,0,1, 1,0,1,1, 0,1,1,1]
    g.reverse()
    curr_code = list(reversed(code)) + [0] * len(data)
    curr_data = [0]* 32 + list(reversed(data))
    for i in range(len(data)):
        if curr_code[i] != curr_data[i]:
            for j in range(33):
                curr_code[i + j] = (curr_code[i + j] + g[j]) % 2
    curr_code.reverse()
    for i in range(32):
        data[i] = curr_code[i]
    # здесь мог бы быть ваш код

**4. Коррекция ошибок.** 
Ошибку, обнаруженную на носителе информации (например, если появилась царапина на DVD), хочется по возможности исправить. В отличие от кодов из предыдущих подзадач, для рассматриваемого кода данные представляются не серией битов, а последовательностью $a_1, a_2, \ldots, a_m$, в которой каждый $a_i$ является элементом поля $\mathbb{Z}_7$ (на практике вместо $\mathbb{Z}_7$ обычно используется поле размера $256$, в котором каждый элемент занимает один байт; о таком поле у нас рассказывается в курсах специальности КБ). Для простоты предположим, что $m$ делится на 5. Данные разбиваются на блоки длины 5: $\{a_{5i+1}, a_{5i+2}, \ldots, a_{5i+5}\}_{i=0}^{m/5-1}$. Блок $b_1, b_2, \ldots, b_5$ ($b_i \in \mathbb{Z}_7$) кодируется последовательностью $c_1, c_2, \ldots, c_7$, в которой каждый элемент $c_i$ для $i \in \{1,\ldots,7\}$ является значением многочлена $\sum_{i=1}^5 b_i x^{5-i}$ в точке $x = i-1$ (вычисления выполняются в $\mathbb{Z}_7$).

Вам нужно написать функцию, которая по числам $c_1, c_2, \ldots, c_7$ возвращает исходные числа $b_1, b_2, \ldots, b_5$, даже если кто-то подменил одно из чисел $c_i$ на другое значение из $\mathbb{Z}_7$ (если изменено больше одного числа $c_i$, вашему коду позволено этого не замечать и вести себя как угодно). Не бойтесь писать переборное решение, сейчас от вас не требуется эффективность, а только корректность и, обязательно, обоснование.

In [33]:
def correct_errors(cblock):
    '''По блоку длины 7 из Z_7 вернуть исходный блок длины 5'''
    b = [0] * 5
    for i in range (7):
        b[0] = i
        for j in range (7):
            b[1] = j
            for k in range (7):
                b[2] = k
                for m in range (7):
                    b[3] = m
                    for n in range (7):
                        b[4] = n
                        if check_result(cblock, b):
                            return b
                        else:
                            continue

def check_result(cblock, b):
    schet = 0
    for t in range(7):
        result = find_sum(b, t)
        if result != cblock[t]:
            schet+=1
    return schet <= 1

def find_sum(b, x):
    xn = 1
    result = 0
    for i in range(4, -1, -1):
        result += (b[i] * xn) % 7
        xn = (xn * x) % 7
    return result % 7  

## Тестирование

Осталось проверить, проходят ли ваши решения первоначальное тестирование. Выполните весь код (Cell -> Run All или Runtime -> Run All или другим способом, работающим в вашей среде) и посмотрите вердикт внизу страницы. В тестирующем коде разбираться не нужно!

In [34]:
import numpy as np, itertools

def test_all():
    data = [1, 0,0,0,0, 0,1,0,0, 1,1,0,0, 0,0,0,1, 0,0,0,1, 1,1,0,1, 1,0,1,1, 0,1,1,1]
    c1 = integrity_code(data)
    data = [1, 0,0,0,0, 0,1,0,0, 1,1,0,0, 0,0,0,1, 0,0,0,1, 1,1,0,1]
    c2 = integrity_code(data)
    data = [1, 0,1,0,0, 0,0,0,1, 1,1,1,0, 0,0,1,0, 0,1,0,1, 1,1,1,0, 1,0,1,0]
    c3 = integrity_code(data)
    if c1 != [0]*32 or c2 != [1,0,1,1,0,1,1,1] + [0]*24 or c3 != [1,1,0,0,0,0,0,1,1,1] + [0]*22:
        print('Первое задание не прошло тест')
    else:
        print('Первое задание прошло все тесты')
    
    print('Тесты для второго задания намеренно не приводятся')
        
    data = [0,1,0,0,0,0,1,0,1,1,0,1,0,0,0,1,1,0,1,1,1,1,1,1,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0]
    data32 = data[0:32]
    code = [0,1,0,0,0,0,1,1,0,1,1,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,1,0,1]
    zeros = [0] * 100
    falsify_integrity_code(data, code)
    falsify_integrity_code(zeros, [0] * 32)
    falsify_integrity_code(data32, [0] * 32)
    expect_data32 =  [1,1,0,1,0,1,0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0,0,0,1,1]
    if data[0:32] != expect_data32 or zeros != [0] * 100 or data32 != [0] * 32:
        print('Третье задание не прошло тест')
    else:
        print('Третье задание прошло все тесты')
        
    cblock = [3,6,0,6,2,0,4]
    block = [2,2,0,6,3]
    succ = True
    for i in range(7):
        cblock[i] = (cblock[i] + 6 + i) % 7
        if correct_errors(cblock) != block:
            succ = False
        cblock[i] = (cblock[i] - 6 - i) % 7
    if succ:
        print('Четвёртое задание прошло все тесты')
    else:
        print('Четвёртое задание не прошло тест')

In [35]:
test_all()

Первое задание прошло все тесты
Тесты для второго задания намеренно не приводятся
Третье задание прошло все тесты
Четвёртое задание прошло все тесты
